# Alice in Wonderland Hyperdictionary Prediction

I am creating a pipeline for testing predictions. I am going to compare different strategies and try and predict the next letter given a sentence from alice and 20 characters within that sentence.



In [1]:

import random_idx
import utils
import pickle
import re
import string

from pylab import *

%matplotlib inline


height has been deprecated.

2016-02-23 13:18


In [4]:
fdict = open("raw_texts/texts_english/alice_in_wonderland.txt")
text = fdict.read()

sentences = text.split('.')

In [5]:
len(sentences)

1207

Next is the function to run the test. This takes in a prediction function, gives it a sentence and asks the function to predict the next letter. Right now, I have the prediction_func return a histogram of letters, but the test_prediction just takes the maximum. Guy was talking about measuring entropy reduction, which is probably a better metric, but this is just a first pass. 

In [ ]:
def test_prediction(prediction_func, lookback=20):
    
    # We're doing all this just to make sure the sentence is long enough
    for i in range(100):
        sidx = np.random.randint(lookback)
        sentence_str = sentences[np.random.randint(len(sentences))].lower()
        
        rm = string.punctuation + string.digits
    
        for p in string.punctuation:
            sentence_str = sentence_str.replace(p, '')
        
        sentence_str = sentence_str.replace('\n',' ')
        sentence_str = sentence_str.replace('\r','')
        sentence_str = sentence_str.replace('\t','')
        sentence_str = sentence_str.strip()
        
        if len(sentence_str[sidx:]) > lookback:
            break
            

    
    # ok, so ask for the next letter
    next_letter_dist = prediction_func(sentence_str[:lookback])
    
    # just take the argmax for now.
    pred_lidx = np.argmax(next_letter_dist)
    
    corr_letter = sentence_str[lookback]
    corr_lidx = random_idx.alphabet.find(corr_letter)
    
    # output to analyze performance
    print sentence_str[:lookback], random_idx.alphabet[corr_lidx], random_idx.alphabet[pred_lidx]
    
    return corr_lidx == pred_lidx
    
    

## Always Guess 'e'

The first thing to try is to just guess 'e' every time. Let's see how that does.

In [46]:
def always_predict_e(sentence):
    letter_hist = zeros(len(random_idx.alphabet))
    
    letter_hist[4] = 1
    
    return letter_hist
    

In [48]:
N = 100
iscorrect_prediction = zeros(N)

for i in range(N):
    iscorrect_prediction[i] = test_prediction(always_predict_e)
    
print np.mean(iscorrect_prediction)

at this moment the k i e
the cook threw a fry i e
hand it over here sa i e
dont grunt said alic e e
when she got back to   e
its enough to drive  o e
they all made a rush   e
if the second copy i s e
they cant have anyth i e
then they all crowde d e
if you are redistrib u e
if you received the  w e
tell her about the r e e
as a duck with its e y e
all the time they we r e
there was no one two   e
now at ours they had   e
well thought alice t o e
it quite makes my fo r e
as if it wasnt troub l e
write that down the  k e
there are a few thin g e
the jury all brighte n e
id rather finish my  t e
so she set to work a n e
how can i have done  t e
the trial cannot pro c e
youre a very poor sp e e
they were just begin n e
the three soldiers w a e
if everybody minded  t e
it exists because of   e
why there they are s a e
dinahll miss me very   e
if youre going to tu r e
in another minute th e e
alice said nothing s h e
they very soon came  u e
the reason is said t h e
unless you have remo v e


So, we can get about 10-15% of the next letter guesses correct by just guessing 'e' every time.

## External Dictionary

A pretty sensible method of predicting the next letter is to use an external dictionary and try and base the guess on the last word. This external dictionary is created from the '2of12id.txt' dictionary, but only contains a subset of the full word list. This dictionary also includes every substring of the word, as well as spaces -- the full word contains a space at the end. This way it can guess space. This dictionary is naive to any of the statistics of the word appearance, and will just guess based on what is possible and not what is most likely.

In [49]:
h = np.load('data/hyperdictionary_external-s20-d1M-160223.npz')
letter_vectors_substr = h['letter_vectors']
hyperdictionary_substr = h['hyperdictionary']

In [50]:
N = hyperdictionary_substr.shape[0]

def predict_from_last_word(sentence):
    # find the last space in the sentence
    words = sentence.split()
    
    last_word = words[-1]    
    
    subword = ''
    subvec = np.ones(N)
    for i,letter in enumerate(last_word):
        letter_idx = random_idx.alphabet.find(letter)
        subvec = np.roll(subvec, 1) * letter_vectors_substr[letter_idx,:]
        subword += letter
        
    subvec = np.roll(subvec, 1)
    
    val = np.dot(letter_vectors_substr/N, subvec*hyperdictionary_substr)
    return val

        
trials = 100
iscorrect_prediction = zeros(trials)

for i in range(trials):
    iscorrect_prediction[i] = test_prediction(predict_from_last_word)
       
print mean(iscorrect_prediction)

either the well was  v t
id rather finish my  t s
please check the pro j f
do not copy display  p f
so they sat down and   a
that you wont though t a
next came the guests   k
it tells the day of  t t
there was a general  c i
she cant explain it  s a
and just as id taken   y
the foundations prin c c
ive tried the roots  o b
silence all round if   r
in a minute or two t h a
we wont talk about h e u
creating the works f r i
begin at the beginni n n
not quite right im a f n
with extras asked th e r
i can tell you more  t  
i cant go no lower s a c
there could be no do u w
when we were little  t  
it was no doubt only   r
alice felt that this   g
here the other guine a k
ah then yours wasnt  a m
the miserable hatter   o
come lets try the fi r l
you must remember re m m
just think of what w o e
hearthrug          n e i
its really dreadful  s n
off with her head th e r
will you wont you wi l c
oh dear what nonsens e u
first because im on  t t
the reason is said t h a
why what are your sh o u


This dictionary is pretty close performance-wise to just guessing 'e'. However, you can see it does a decent job when there is a long word.

## Using internal hyperdictionary

Next, I built a similar substring hyperdictionary as before, but this time I used the list of words actually from alice. This is a pretty ideal dictionary to have handy. The performance of this dictionary is useful to compare with other algorithms, as this will have a good chance of working well. If another algorithm can beat this, then it has learned a lot about english and predicting Alice, and probably has an important insight about learning.

In [52]:
h = np.load('data/hyperdictionary_alice-d1M-160223.npz')
letter_vectors_alice = h['letter_vectors']
hyperdictionary_alice = h['hyperdictionary']

In [53]:
N = hyperdictionary_alice.shape[0]

def predict_from_last_word(sentence):
    # find the last space in the sentence
    words = sentence.split()
    
    last_word = words[-1]    
    
    subword = ''
    subvec = np.ones(N)
    for i,letter in enumerate(last_word):
        letter_idx = random_idx.alphabet.find(letter)
        subvec = np.roll(subvec, 1) * letter_vectors_alice[letter_idx,:]
        subword += letter
        
    subvec = np.roll(subvec, 1)
    
    val = np.dot(letter_vectors_alice/N, subvec*hyperdictionary_alice)
    return val

        
trials = 100
iscorrect_prediction = zeros(trials)

for i in range(trials):
    iscorrect_prediction[i] = test_prediction(predict_from_last_word)
       
print mean(iscorrect_prediction)

3 this work is provi d s
the first thing she  h s
so she swallowed one    
alice kept her eyes  a  
nobody asked your op i p
are you content now  s  
how could he turn th e e
it was the white rab b b
she hastily put down    
here one of the guin e e
is that the reason s o l
i know what it means    
are you content now  s  
then you may sit dow n n
how fond she is of f i e
its 501c3 letter is  p n
international donati o o
at last the gryphon  s  
i only took the regu l l
alices evidence   he r i
are you to get in at    
i dont see said the  c m
oh i know exclaimed  a  
let me see four time s s
she cant explain it  s s
the jury all wrote d o r
and how did you mana g g
hadnt time said the  g m
do you mean that you   t
at last the gryphon  s  
alice said nothing s h l
the hatter was the f i e
its really dreadful  s  
i couldnt afford to  l r
the duchess took no  n r
and then turning to  t r
but what happens whe n r
you may copy it give   n
except for the limit e e
leave off that screa m m


In [54]:
trials = 100
iscorrect_prediction = zeros(trials)

for i in range(trials):
    iscorrect_prediction[i] = test_prediction(predict_from_last_word)
       
print mean(iscorrect_prediction)

not at all said alic e e
for you see so many  o  
imagine her surprise    
with extras asked th e e
however she got up a n m
who are you talking  t  
i dont believe there s  
the jury all looked  p  
indemnity  you agree   m
its 501c3 letter is  p n
if you didnt sign it   s
he was an old crab h e j
unless a copyright n o o
i keep them to sell  t  
so alice began telli n n
alice was very nearl y y
turn a somersault in   j
i should like to hea r p
i thought it would s a l
here the other guine a a
certainly not said a l m
of course we hope th a e
i hope theyll rememb e e
leave off that screa m m
i do alice hastily r e o
why you dont even kn o i
not the same thing a   m
tell us a story said    
and so these three l i a
i must go and get re a v
i could tell you my  a s
of course we hope th a e
no no the adventures    
is that the way you  m t
i wonder what theyll    
they couldnt have do n i
why said the dodo th e e
what trial is it ali c c
and how did you mana g g
would you like cats  i  
